In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# 크롤링 함수 정의
def crawl_company_info(code):
    url = f"https://comp.fnguide.com/SVO2/asp/SVD_Corp.asp?pGB=1&gicode={code}"
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(2)

    data = []

    def extract(xpath):
        try:
            rows = driver.find_elements(By.XPATH, xpath)
            for row in rows:
                th = row.find_element(By.TAG_NAME, 'th').text.strip()
                td = row.find_element(By.TAG_NAME, 'td').text.strip()
                data.append([th, td])
        except:
            pass

    extract('//*[@id="corpGeneralInfo"]/table/tbody/tr')          # 주소
    extract('//*[@id="compBody"]/div[4]/div[1]/table/tbody/tr')   # 대표자, 설립일 등
    extract('//*[@id="compBody"]/div[4]/div[2]/table/tbody/tr')   # 홈페이지, 전화번호

    driver.quit()
    return pd.DataFrame(data, columns=["항목", "값"]).set_index("항목").T

# 예시 종목코드 리스트
sample_codes = ['A005930', 'A000660', 'A035420', 'A068270', 'A207940']

# 크롤링 실행
dfs = []
for code in sample_codes:
    df = crawl_company_info(code)
    df["종목코드"] = code
    dfs.append(df)

# 결과 통합 및 저장
final_df = pd.concat(dfs, ignore_index=True)
final_df.to_csv("기업개요_샘플5개.csv", index=False, encoding='utf-8-sig')
print("✅ 저장 완료: 기업개요_샘플5개.csv")


✅ 저장 완료: 기업개요_샘플5개.csv


In [ ]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


filepath = r"C:\Users\bitcamp\Downloads\Python\DART재무_코스피전체_2024기준 (1).xlsx"
df = pd.read_excel(file_path)

codes = df[['종목코드', '종목명']].dropna()
codes['종목코드'] = codes['종목코드'].astype(str).str.zfill(6)

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

results = []
total = len(codes)

for i, row in codes.iterrows():
    code = row['종목코드']
    name = row['종목명']
    url = f"https://comp.fnguide.com/SVO2/asp/SVD_Corp.asp?pGB=1&gicode=A{code}"

    try:
        driver.get(url)
        time.sleep(2)
        element = driver.find_element(By.XPATH, '//*[@id="corpGeneralInfo"]')
        text = element.text.strip()
        status = "성공"
    except Exception as e:
        text = f"오류 발생: {e}"
        status = "실패"

    results.append({
        '종목코드': code,
        '종목명': name,
        '기업개요': text
    })

    print(f"[{i+1}/{total}] {code} - {name} → {status}")

driver.quit()

output_path = r"C:\Users\bitcamp\Downloads\Python기업설명기업개요전체.csv"
result_df = pd.DataFrame(results)
result_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 전체 크롤링 완료! 결과 저장 위치:\n{output_path}")

In [8]:
df=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\기업개요전체.csv')
import pandas as pd
import re


df=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\기업개요전체.csv')

# ▶ 필드 추출 함수
def extract_field(text, field):
    try:
        match = re.search(fr'{field}\n([^\n]+)', str(text))
        return match.group(1).strip() if match else None
    except:
        return None

# ▶ 전화번호 및 홈페이지 정규표현식 추출
def extract_phone(text):
    match = re.search(r'(\d{2,4}-\d{3,4}-\d{4})', str(text))
    return match.group(1) if match else None

def extract_homepage(text):
    match = re.search(r'(https?://[^\s\n]+)', str(text))
    return match.group(1) if match else None

# ▶ 주요 항목 컬럼 생성
df['주소'] = df['기업개요'].apply(lambda x: extract_field(x, '주소'))
df['설립일'] = df['기업개요'].apply(lambda x: extract_field(x, '설립일'))
df['대표자'] = df['기업개요'].apply(lambda x: extract_field(x, '대표자'))
df['전화번호'] = df['기업개요'].apply(extract_phone)
df['홈페이지'] = df['기업개요'].apply(extract_homepage)
df['설립연도'] = df['설립일'].apply(lambda x: str(x).split('/')[0] if pd.notnull(x) else None)

# ▶ 결과 저장
output_path = "기업개요_정리완료.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 기업개요 정리 완료: {output_path}")


✅ 기업개요 정리 완료: 기업개요_정리완료.csv


In [14]:
df=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\기업개요.csv')
df

,종목코드,종목명,주소,설립일,대표자,전화번호,홈페이지
0,95570,AJ네트웍스,서울시 송파구 정의로 8길 9 (AJ빌딩),2000-02-10,곽정호,02-6240-0800,http://www.ajnetworks.co.kr
1,6840,AK홀딩스,서울시 마포구 양화로 188(동교동),1970-10-13,"고준, 채형석 (각자 대표)",02-6923-2921,http://www.aekyunggroup.co.kr
2,27410,BGF,서울시 강남구 테헤란로 405(삼성동),1994-12-01,홍정국,NaN,http://www.bgf.co.kr
3,282330,BGF리테일,서울시 강남구 테헤란로 405 BGF사옥,2017-11-01,민승배,02-528-6829,http://www.bgfretail.com
4,138930,BNK금융지주,부산시 남구 문현금융로 30 (문현동),2011-03-15,빈대인,051-620-3000,http://www.bnkfg.com
...,...,...,...,...,...,...,...
843,5870,휴니드테크놀러지스,인천시 연수구 벤처로 87,1968-12-11,김유진,032-457-6000,http://www.huneed.com
844,79980,휴비스,"서울시 강남구 학동로 343, POBA강남타워 12층",2000-11-01,김석현,02-2189-4567,http://www.huvis.com
845,5010,휴스틸,"서울시 강남구 테헤란로 512, 14층 (대치동, 신안빌딩)",1967-04-18,"박훈, 윤형원(각자 대표)",02-828-9000,http://www.husteel.com
846,540,흥국화재,서울시 종로구 새문안로 68,1948-03-15,송윤상,02-2002-6000,http://www.heungkukfire.co.kr
